In [1]:
!pip uninstall -y transformers accelerate peft trl bitsandbytes unsloth
!pip install -q \
  unsloth \
  transformers \
  accelerate \
  peft \
  trl \
  bitsandbytes \
  datasets


Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 135.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.3 MB/s eta 0:00:

In [2]:
!pip install psutil

In [4]:
!pip install trl==0.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.24.0
    Uninstalling trl-0.24.0:
      Successfully uninstalled trl-0.24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.12.9 requires trl!=0.19.0,<=0.24.0,>=0.18.2, but you have trl 0.14.0 which is incompatible.
unsloth-zoo 2025.12.7 requires trl!=0.19.0,<=0.24.0,>=0.18.2, but you have trl 0.14.0 which is incompatible.


In [1]:
import torch
from unsloth import FastLanguageModel

model_name = "unsloth/Meta-Llama-3.1-8B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 512,
    dtype = None,            # auto
    load_in_4bit = True,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not find `steps_per_generation` in grpo_trainer
Unsloth: Could not find `generation_batch_size` in grpo_trainer
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [3]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="data-myself.json",
    split="train",
)


Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
dataset[1]

{'question': 'What year the user was born?',
 'answer': 'The user was born in 2004'}

In [7]:
def formatting_func(example):
    text = (
        "### Question:\n"
        f"{example['question']}\n\n"
        "### Answer:\n"
        f"{example['answer']}"
    )

    # tokenize FIRST
    tokens = tokenizer(
        text,
        truncation=True,
        max_length=512,
        add_special_tokens=True,
    )

    # decode back to text
    text = tokenizer.decode(tokens["input_ids"], skip_special_tokens=False)

    return [text]



In [5]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./lora_out",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=100,
    learning_rate=2e-4,
    fp16=True,                 # NOT bf16 (avoids your crash)
    logging_steps=1,
    save_steps=50,
    optim="adamw_8bit",
    report_to="none",
)
training_args.dataset_num_proc = 1

In [8]:

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    processing_class=tokenizer,
    formatting_func=formatting_func,
    max_seq_length=512,
    packing=False,   # 👈 important for debugging
)


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [9]:
trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1 | Num Epochs = 100 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488 of 8,043,892,736 (0.17% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.992400
2,1.992400
3,1.988500
4,1.972700
5,1.945800
6,1.906400
7,1.852400
8,1.781800
9,1.695000
10,1.589900


TrainOutput(global_step=100, training_loss=0.27653646471910176, metrics={'train_runtime': 164.4979, 'train_samples_per_second': 4.863, 'train_steps_per_second': 0.608, 'total_flos': 2309700452352000.0, 'train_loss': 0.27653646471910176, 'epoch': 100.0})

In [10]:
model.save_pretrained("lora_adapter")
tokenizer.save_pretrained("lora_adapter")


('lora_adapter/tokenizer_config.json',
 'lora_adapter/special_tokens_map.json',
 'lora_adapter/chat_template.jinja',
 'lora_adapter/tokenizer.json')

In [11]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [24]:
question = "What languages does the user speak"

prompt = f"""### Question:
{question}

### Answer:
"""

inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=20,          # enough for a year like "2004"
    do_sample=False,           # deterministic output
    temperature=0.0,           # no randomness
    eos_token_id=tokenizer.eos_token_id
)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract only the answer part
answer = decoded.split("### Answer:")[-1].strip().splitlines()[0]

print(answer)


The user speaks English and Japanese.
